In [ ]:
# pip install llama-index llama-index-embeddings-openai llama-index-graph-stores-kuzu

[Kùzu](https://kuzudb.com/) is an open source, embedded graph database that's designed for query speed and scalability. It implements the Cypher query language, and utilizes a structured property graph model (a variant of the labelled property graph model) with support for ACID transactions. Because Kùzu is embedded, there's no requirement for a server to set up and use the database.

Let's begin by creating a graph from unstructured text to demonstrate how to use Kùzu as a graph and vector store to answer questions.

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Environment Setup

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

We will be using OpenAI models for this example, so we'll specify the OpenAI API key.

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-08-06 13:30:12--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.004s  

2025-08-06 13:30:12 (19.3 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

## Graph Construction

We first need to create an empty Kùzu database directory by calling the `kuzu.Database` constructor. This step instantiates the database and creates the necessary directories and files within a local directory that stores the graph. This `Database` object is then passed to the `KuzuPropertyGraph` constructor.

In [ ]:
from pathlib import Path
import kuzu

DB_NAME = "ex.kuzu"
Path(DB_NAME).unlink(missing_ok=True)
db = kuzu.Database(DB_NAME)

Because Kùzu implements the structured graph property model, it imposes some level of structure on the schema of the graph. In the above case, because we did not specify a relationship schema that we want in our graph, it uses a generic schema, where the relationship types are not constrained, allowing the extracted triples from the LLM to be stored as relationships in the graph.

### Define LLMs

Below, we'll define the models used for embedding the text and the LLMs that are used to extract triples from the text and generate the response.
In this case, we specify different temperature settings for the same model - the extraction model has a temperature of 0.

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")
extract_llm = OpenAI(model="gpt-4.1-mini", temperature=0.0)
generate_llm = OpenAI(model="gpt-4.1-mini", temperature=0.3)

## Create property graph index with structure

The recommended way to use Kùzu is to apply a structured schema to the graph. The schema is defined by specifying the relationship types (including direction) that we want in the graph. The imposition of structure helps with generating triples that are more meaningful for the types of questions we may want to answer from the graph.

By specifying the below validation schema, we can enforce that the graph only contains relationships of the specified types.

In [ ]:
from typing import Literal

entities = Literal["PERSON", "PLACE", "ORGANIZATION"]
relations = Literal["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"]
# Define the relationship schema that we will pass to our graph store
# This must be a list of valid triples in the form (head_entity, relation, tail_entity)
validation_schema = [
    ("ORGANIZATION", "HAS", "PERSON"),
    ("PERSON", "WORKED_AT", "ORGANIZATION"),
    ("PERSON", "WORKED_WITH", "PERSON"),
    ("PERSON", "WORKED_ON", "ORGANIZATION"),
    ("PERSON", "PART_OF", "ORGANIZATION"),
    ("ORGANIZATION", "PART_OF", "ORGANIZATION"),
    ("PERSON", "WORKED_AT", "PLACE"),
]

## Create property graph store with a vector index

To create a `KuzuPropertyGraphStore` with a vector index, we need to specify the `use_vector_index` parameter as `True`. This will create a vector index on the property graph, allowing us to perform vector-based queries on the graph.

In [ ]:
from llama_index.graph_stores.kuzu import KuzuPropertyGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

graph_store = KuzuPropertyGraphStore(
    db,
    has_structured_schema=True,
    relationship_schema=validation_schema,
    use_vector_index=True,  # Enable vector index for similarity search
    embed_model=embed_model,  # Auto-detects embedding dimension from model
)

Auto-detected embedding dimension: 1536


To construct a property graph with the desired schema, we'll use `SchemaLLMPathExtractor` with the following parameters.

In [ ]:
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=embed_model,
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=extract_llm,
            possible_entities=entities,
            possible_relations=relations,
            kg_validation_schema=validation_schema,
            strict=True,  # if false, will allow triples outside of the schema
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

/Users/prrao/code/llama_index/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 4/4 [00:01<00:00,  3.59it/s]


We can now apply the query engine on the index as before.

In [ ]:
# Switch to the generate LLM during retrieval
Settings.llm = generate_llm

query_text = "Tell me more about Interleaf and Viaweb?"
query_engine = index.as_query_engine(include_text=False)

response = query_engine.query(query_text)
print(str(response))

The information provided mentions Viaweb and associates it with individuals named Trevor, Trevor Blackwell, Robert, and Paul Graham. However, there is no information given about Interleaf or further details about Viaweb beyond these associations.


In [ ]:
retriever = index.as_retriever(include_text=False)
nodes = retriever.retrieve(query_text)
nodes[0].text

'Viaweb -> HAS -> Trevor'

## Query the vector index

As an embedded graph database, Kuzu provides a fast and performance graph-based HNSW vector index (see the [docs](https://docs.kuzudb.com/extensions/vector/)).
This allows you to also use Kuzu for similarity (vector-based) retrieval on chunk nodes.
The vector index is created after the embeddings are ingested into the chunk nodes, so you should be able to query them directly.

In [ ]:
from llama_index.core.vector_stores.types import VectorStoreQuery

query_text = "How much funding did Idelle Weber provide to Viaweb?"
query_embedding = embed_model.get_text_embedding(query_text)
# Perform direct vector search on the graph store
vector_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

nodes, similarities = graph_store.vector_query(vector_query)

for i, (node, similarity) in enumerate(zip(nodes, similarities)):
    print(f"  {i + 1}. Similarity: {similarity:.3f}")
    print(f"     Text: {node.text}...")
    print()

  1. Similarity: 0.421
     Text: Large numbers of former students kept in touch with her, including me. After I moved to New York I became her de facto studio assistant.

She liked to paint on big, square canvases, 4 to 5 feet on a side. One day in late 1994 as I was stretching one of these monsters there was something on the radio about a famous fund manager. He wasn't that much older than me, and was super rich. The thought suddenly occurred to me: why don't I become rich? Then I'll be able to work on whatever I want.

Meanwhile I'd been hearing more and more about this new thing called the World Wide Web. Robert Morris showed it to me when I visited him in Cambridge, where he was now in grad school at Harvard. It seemed to me that the web would be a big deal. I'd seen what graphical user interfaces had done for the popularity of microcomputers. It seemed like the web would do the same for the internet.

If I wanted to get rich, here was the next train leaving the station. I was rig

In [ ]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import QueryBundle, NodeWithScore, TextNode
from llama_index.core.vector_stores.types import VectorStoreQuery
from llama_index.core.indices.property_graph import LLMSynonymRetriever
from typing import List


class GraphVectorRetriever(BaseRetriever):
    """
    A retriever that performs vector search on a property graph store.
    """

    def __init__(self, graph_store, embed_model, similarity_top_k: int = 5):
        self.graph_store = graph_store
        self.embed_model = embed_model
        self.similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # Get query embedding
        query_embedding = self.embed_model.get_text_embedding(
            query_bundle.query_str
        )

        # Perform vector search
        vector_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self.similarity_top_k,
        )
        nodes, similarities = self.graph_store.vector_query(vector_query)

        # Convert ChunkNodes to TextNodes
        nodes_with_scores = []
        for node, similarity in zip(nodes, similarities):
            # Convert ChunkNode to TextNode
            if hasattr(node, "text"):
                text_node = TextNode(
                    text=node.text,
                    id_=node.id,
                    metadata=getattr(node, "properties", {}),
                )
                nodes_with_scores.append(
                    NodeWithScore(node=text_node, score=similarity)
                )

        return nodes_with_scores


class CombinedGraphRetriever(BaseRetriever):
    """
    A retriever that performs that combines graph and vector search on a property graph store.
    """

    def __init__(
        self, graph_store, embed_model, llm, similarity_top_k: int = 5
    ):
        self.graph_store = graph_store
        self.embed_model = embed_model
        self.llm = llm
        self.similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # 1. Vector retrieval
        query_embedding = self.embed_model.get_text_embedding(
            query_bundle.query_str
        )
        vector_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self.similarity_top_k,
        )
        vector_nodes, similarities = self.graph_store.vector_query(
            vector_query
        )

        # Convert ChunkNodes to TextNodes for vector results
        vector_results = []
        for node, similarity in zip(vector_nodes, similarities):
            if hasattr(node, "text"):
                text_node = TextNode(
                    text=node.text,
                    id_=node.id,
                    metadata=getattr(node, "properties", {}),
                )
                vector_results.append(
                    NodeWithScore(node=text_node, score=similarity)
                )

        # 2. Graph traversal retrieval
        graph_retriever = LLMSynonymRetriever(
            self.graph_store, llm=self.llm, include_text=True
        )
        graph_results = graph_retriever.retrieve(query_bundle)

        # 3. Combine and deduplicate
        all_results = vector_results + graph_results
        seen_nodes = set()
        combined_results = []

        for node_with_score in all_results:
            node_id = node_with_score.node.node_id
            if node_id not in seen_nodes:
                seen_nodes.add(node_id)
                combined_results.append(node_with_score)

        return combined_results


# Use the combined retriever
combined_retriever = CombinedGraphRetriever(
    graph_store=graph_store,
    llm=generate_llm,
    embed_model=embed_model,
    similarity_top_k=5,
)

# Test the combined retriever
query_text = "What was the role of Idelle Weber in Viaweb?"
query_bundle = QueryBundle(query_str=query_text)
results = combined_retriever.retrieve(query_bundle)
for i, node_with_score in enumerate(results):
    print(f"{i + 1}. Score: {node_with_score.score:.3f}")
    print(
        f"   Text: {node_with_score.node.text[:100]}..."
    )  # Print first 100 chars
    print(f"   Node ID: {node_with_score.node.node_id}")
    print()

1. Score: 0.371
   Text: Large numbers of former students kept in touch with her, including me. After I moved to New York I b...
   Node ID: 48bda642-e94d-4b79-96fc-4f92ab8813c3

2. Score: 0.353
   Text: I'd compounded this problem by buying a house up in the Santa Cruz Mountains, with a beautiful view ...
   Node ID: 17e8d852-4c00-4eab-8258-641b074c8abb

3. Score: 0.314
   Text: In its time, the editor was one of the best general-purpose site builders. I kept the code tight and...
   Node ID: 1c666a5d-9058-42cf-8369-b21749b401b4

4. Score: 0.298
   Text: I started working on the application builder, Dan worked on network infrastructure, and the two unde...
   Node ID: 9e8a378c-015a-4184-a66b-b00939e11a31

5. Score: 0.284
   Text: The UI was horrible, but it proved you could build a whole store through the browser, without any cl...
   Node ID: 9f4b9b54-7be7-4d18-8689-493729dd9078

6. Score: 1.000
   Text: Paul Graham -> WORKED_WITH -> McCarthy...
   Node ID: b9d67e62-994c-416d-835b-3f

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import get_response_synthesizer

# Create query engine with your combined retriever
query_engine = RetrieverQueryEngine.from_args(
    retriever=combined_retriever,
    llm=generate_llm,
)

# Create response synthesizer
response_synthesizer = get_response_synthesizer(
    llm=generate_llm, use_async=False
)

# Create query engine
query_engine = RetrieverQueryEngine(
    retriever=combined_retriever, response_synthesizer=response_synthesizer
)

# Query and get answer
query_text = "What was the role of Idelle Weber in Viaweb?"
response = query_engine.query(query_text)
print(response)

Idelle Weber was connected to Viaweb through her husband Julian, who provided $10,000 in seed funding for the company. In return for this funding, legal work, and business advice, they gave him 10% of the company.
